In [1]:
import numpy as np
rng = np.random.RandomState(42)
x = rng.rand(1000000)
y = rng.rand(1000000)
%timeit x + y

1.6 ms ± 61.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [3]:
%timeit np.fromiter((xi + yi for xi, yi in zip(x, y)), dtype=x.dtype, count=len(x))

279 ms ± 4.96 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
mask = (x > 0.5) & (y < 0.5)

In [5]:
tmp1 = (x>0.5)
tmp2 = (y<0.5)
mask = tmp1 & tmp2

In [6]:
import numexpr

mask_numexpr = numexpr.evaluate('(x>0.5) & (y<0.5)')
np.allclose(mask, mask_numexpr)

True

In [7]:
import pandas as pd

nrows, ncols = 100000, 100
rng = np.random.RandomState(42)
df1, df2, df3, df4 = (pd.DataFrame(rng.rand(nrows, ncols))
                     for i in range(4))

/home/arti/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [8]:
%timeit df1 + df2 + df3 + df4

181 ms ± 1.91 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
%timeit pd.eval('df1 + df2 + df3 + df4')

58.8 ms ± 524 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
np.allclose(df1 + df2 + df3 + df4,
           pd.eval('df1 + df2 + df3 + df4'))

True

In [11]:
df1, df2, df3, df4, df5 = (pd.DataFrame(rng.randint(0, 1000, (100, 3)))
                          for i in range(5))

In [12]:
result1 = -df1 * df2 / (df3 + df4) - df5
result2 = pd.eval('-df1 * df2 / (df3 + df4) - df5')
np.allclose(result1, result2)

True

In [13]:
df1

,0,1,2
0,180,112,748
1,447,205,487
2,656,100,98
3,90,450,613
4,529,224,530
...,...,...,...
95,31,787,643
96,984,624,352
97,283,543,751
98,5,142,278


In [15]:
result1 = (df1 < df2) & (df2 <= df3) & (df3 != df4)
result2 = pd.eval('df1 < df2 <= df3 != df4')
np.allclose(result1, result2)

True

In [16]:
result1 = (df1 < 0.5) & (df2 < .5) | (df3 < df4)
result2 = pd.eval('(df1 < .5) & (df2 < .5) | (df3 < df4)')
np.allclose(result1, result2)

True

In [17]:
result3 = pd.eval('(df1 < .5) and (df2 < .5) or (df3 < df4)')
np.allclose(result1, result3)

True

In [18]:
result1 = df2.T[0] + df3.iloc[1]
result2 = pd.eval('df2.T[0] + df3.iloc[1]')
np.allclose(result1, result2)

True

In [19]:
df = pd.DataFrame(rng.rand(1000, 3), columns=['A', 'B', 'C'])
df.head()

,A,B,C
0,0.375506,0.406939,0.069938
1,0.069087,0.235615,0.154374
2,0.677945,0.433839,0.652324
3,0.264038,0.808055,0.347197
4,0.589161,0.252418,0.557789


In [21]:
result1 = (df['A'] + df['B']) / (df['C']-1)
result2 = pd.eval("(df.A + df.B) / (df.C - 1)")
np.allclose(result1, result2)

True

In [23]:
result3 = df.eval('(A + B) / (C - 1)')
np.allclose(result1, result3)

True

In [24]:
df.head()

,A,B,C
0,0.375506,0.406939,0.069938
1,0.069087,0.235615,0.154374
2,0.677945,0.433839,0.652324
3,0.264038,0.808055,0.347197
4,0.589161,0.252418,0.557789


In [25]:
df.eval('D = (A + B) / C', inplace=True)
df.head()

,A,B,C,D
0,0.375506,0.406939,0.069938,11.187620
1,0.069087,0.235615,0.154374,1.973796
2,0.677945,0.433839,0.652324,1.704344
3,0.264038,0.808055,0.347197,3.087857
4,0.589161,0.252418,0.557789,1.508776


In [26]:
df.eval('D = (A - B) / C', inplace=True)
df.head()

,A,B,C,D
0,0.375506,0.406939,0.069938,-0.449425
1,0.069087,0.235615,0.154374,-1.078728
2,0.677945,0.433839,0.652324,0.374209
3,0.264038,0.808055,0.347197,-1.566886
4,0.589161,0.252418,0.557789,0.603708


In [27]:
column_mean = df.mean(1)
result1 = df['A'] + column_mean
result2 = df.eval('A + @column_mean')
np.allclose(result1, result2)

True

In [28]:
result1 = df[(df.A < .5) & (df.B < .5)]
result2 = pd.eval('df[(df.A < .5) & (df.B < .5)]')
np.allclose(result1, result2)

True

In [31]:
result3 = df.query('A < .5 and B < .5')
np.allclose(result1, result3)

True

In [32]:
Cmean = df['C'].mean()
result1 = df[(df.A < Cmean) & (df.B < Cmean)]
result2 = df.query('A < @Cmean and B < @Cmean')
np.allclose(result1, result2)

True